# Tutorial: Creating a Generative QA Pipeline with PromptNode
From the Haystack [tutorial](https://haystack.deepset.ai/tutorials/22_pipeline_with_promptnode)

In [ ]:
import pathlib

from datasets import load_dataset
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import EmbeddingRetriever, BM25Retriever, PromptTemplate, PromptNode
from haystack.pipelines import FAQPipeline, Pipeline

In [ ]:
DATA_DIR = pathlib.Path().absolute().parent / "data"

In [ ]:
document_store = InMemoryDocumentStore(use_bm25=True)

In [ ]:
dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
document_store.write_documents(dataset)

In [ ]:
retriever = BM25Retriever(document_store=document_store, top_k=2)

In [ ]:
lfqa_prompt = PromptTemplate(
    name="lfqa",
    prompt_text=
    """
    Synthesize a comprehensive answer from the following text for the given question.
    Provide a clear and concise response that summarizes the key points and information presented in the text.
    Your answer should be in your own words and be no longer than 50 words.
    \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
)

prompt_node = PromptNode(model_name_or_path="google/flan-t5-large", default_prompt_template=lfqa_prompt)


In [ ]:
pipe = Pipeline()
pipe.add_node(component=retriever, name="retriever", inputs=["Query"])
pipe.add_node(component=prompt_node, name="prompt_node", inputs=["retriever"])

In [ ]:
output = pipe.run(query="How does Rhodes Statue look like?")

print(output["results"])

In [ ]:
examples = [
    "Where is Gardens of Babylon?",
    "Why did people build Great Pyramid of Giza?",
    "How does Rhodes Statue look like?",
    "Why did people visit the Temple of Artemis?",
    "What is the importance of Colossus of Rhodes?",
    "What happened to the Tomb of Mausolus?",
    "How did Colossus of Rhodes collapse?",
]
for q in examples:
    output = pipe.run(query=q)
    print(q, "\n", output["results"], "\n")